<a href="https://colab.research.google.com/github/zsucicdl/lumen/blob/master/lumen_cl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt install libomp-dev
!python -m pip install --upgrade faiss faiss-gpu
import faiss

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  libomp5
Suggested packages:
  libomp-doc
The following NEW packages will be installed:
  libomp-dev libomp5
0 upgraded, 2 newly installed, 0 to remove and 34 not upgraded.
Need to get 239 kB of archives.
After this operation, 804 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libomp5 amd64 5.0.1-1 [234 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libomp-dev amd64 5.0.1-1 [5,088 B]
Fetched 239 kB in 1s (413 kB/s)
Selecting previously unselected package libomp5:amd64.
(Reading database ... 160690 files and directories currently installed.)
Preparing to unpack .../libomp5_5.0.1-1_amd64.deb ...
Unpacking libomp5:amd64 (5.0.1-1) ...
Se

In [ ]:
import pickle
import os
import time
import numpy as np
import pandas as pd
from sklearn import model_selection
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler
import pickle
from sklearn import metrics
import matplotlib.pyplot as plt
from sklearn.datasets import make_blobs
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler
#from kneed import KneeLocator
from google.colab import drive
import scipy

In [ ]:
drive.mount('/content/drive')
data_path = "/content/drive/MyDrive/LUMEN0.pkl"
with open(data_path, 'rb') as f:
    df = pickle.load(f)
df.shape

Mounted at /content/drive


(1294962, 33)

In [ ]:
# List of all the features
for feature_name in df.keys():
    print(feature_name)

Manufacturing Region
Manufacturing Location Code
Intercompany
CustomerID
Customer industry
Customer Region
Customer First Invoice Date
Top Customer Group
Item Code
Product family
Product group
Price last modified date in the ERP
Born on date
Make vs Buy
Sales Channel - Internal
Sales Channel - External
Sales Channel - Grouping
Invoice Date
Invoice #
Invoice Line #
Order Date
Order #
Order Line #
Invoiced qty (shipped)
Ordered qty
Invoiced price
Invoiced price (TX)
Cost of part
Material cost of part
Labor cost of part
Overhead cost of part
GM%
# of unique products on a quote


In [ ]:
df.drop(
    ['CustomerID',
     'Item Code',
     'Invoice #',
     'Order #',
     'Invoiced price (TX)'],
    axis=1, inplace=True)

In [ ]:
# Invoiced price, should be > 0
df.drop(df[df['Invoiced price'] <= 0].index, inplace=True)

In [ ]:
df = df[~df['Invoiced price'].isnull()]

# df = df[~df['GM%'].isnull()]
# df = df[df['GM%'] >= -1]  # ???
# df = df[df['GM%'] <= 1]  # ???
#df.dropna(inplace=True)

#0 je po meni legalna vrijednost za invoiced quantity, ali ne i za ordered
df = df[df['Invoiced qty (shipped)'] >= 0]
df = df[df['Invoiced qty (shipped)'] < 1000000]
df = df[df['Ordered qty'] > 0]
df = df[df['Ordered qty'] < 1000000]
df = df[df['Invoiced price'] > 0]
df = df[df['Invoiced price'] < 100000]
df = df[df['Cost of part'] > 0]
df = df[df['Cost of part'] < 75000]

df = df[~df['# of unique products on a quote'].isnull()]
df = df[df['# of unique products on a quote'] < 374]
df = df[df['# of unique products on a quote'] > 0]

#simple drop
df = df[~df['Product group'].isnull()]
df = df[~df['Manufacturing Location Code'].isnull()]
df = df[~df['Make vs Buy'].isnull()]
df = df[~df['Customer industry'].isnull()]

#ovo nije dobro, maknuo sam ih jer ih je malo. Jel se ovdje moze koristiti SMOTE?
df = df[df['Product family']!='PC010']
df = df[df['Product family']!='PC001']
df = df[df['Product family']!='PC016']

df = df[df['Product family']!='PF000']

# df[['Material cost of part',
#     'Labor cost of part',
#     'Overhead cost of part'
#     ]] = df[['Material cost of part',
#              'Labor cost of part',
#              'Overhead cost of part'
#              ]].fillna(value=0)
# df['sum'] = df['Material cost of part'] + df['Labor cost of part'] + df['Overhead cost of part']
# df['sum'] = np.where(df['sum']==0, df['Cost of part'], df['sum'])

df['Invoiced qty (shipped)'].astype(int)
df['Ordered qty'].astype(int)
df['# of unique products on a quote'].astype(int)

1          3
11         1
12         1
13         1
14         1
          ..
1294955    6
1294956    5
1294958    1
1294959    1
1294960    1
Name: # of unique products on a quote, Length: 819364, dtype: int64

In [ ]:
a = [
    'Manufacturing Region',
    'Intercompany',
    'Customer industry',
    'Customer Region',
    'Top Customer Group',
    'Product family',
    'Make vs Buy'
]
df.drop(a, axis=1, inplace=True)

In [ ]:
cdf = df.drop(
    ['Manufacturing Location Code',
     'Born on date',
     'Customer First Invoice Date',
     'Born on date',
     'Invoice Date',
     'Invoice Line #',
     'Order Date',
     'Order Line #',
     'Invoiced qty (shipped)',
     'Product group', 
     'Sales Channel - Internal', 
     'Sales Channel - External',
     '# of unique products on a quote',
     'Material cost of part', 
     'Labor cost of part', 
     'Overhead cost of part','Price last modified date in the ERP', 
     'Sales Channel - Grouping'],
    axis=1, inplace=False)
# cdf = pd.get_dummies(cdf)
# cdf

In [ ]:
df.shape

(819364, 21)

In [ ]:
cdf.columns

Index(['Ordered qty', 'Invoiced price', 'Cost of part', 'GM%'], dtype='object')

In [ ]:
noto = (cdf['GM%'] > 1) | (cdf['GM%'] < -1)
cdf = cdf[~noto]
for feature_name in ['Ordered qty', 'Invoiced price', 'Cost of part']:
    cdf.drop(index=cdf[(cdf[feature_name] <= 0)].index, inplace=True)
    feature = cdf[feature_name]
    feature = np.log(feature)
    feature = (feature - feature.mean()) / feature.std()
    cdf[feature_name] = feature
cdf

,Ordered qty,Invoiced price,Cost of part,GM%
1,0.380488,-0.880580,-1.315969,0.816429
11,-0.886879,1.738345,1.649255,0.526436
12,-0.886879,1.734725,1.649255,0.521512
16,0.524261,0.146940,0.127823,0.409060
17,0.524261,0.649574,0.675792,0.332513
...,...,...,...,...
1294955,-0.843073,0.815855,0.789733,0.426160
1294956,-0.748696,1.064649,1.066684,0.380507
1294958,-0.997262,0.488459,-0.129498,0.892857
1294959,1.135871,-1.122154,-1.118960,0.358974


In [ ]:
1-len(cdf) / len(df)

0.023026396082815426

In [ ]:
cdf.shape

(800497, 4)

In [ ]:
#import plotly.express as px
#column = 'Invoiced price'

#fig = px.histogram(df, x=column, log_y=True)
#fig.show()

#cdf=cdf[0:1000]

In [ ]:
# from sklearn.cluster import OPTICS

# cluster = OPTICS(n_jobs=-1,max_eps=5)
# cluster.fit(cdf)

In [ ]:

# kmeans = KMeans(n_clusters=500)
# kmeans.fit(cdf)
# kmeans.labels_

In [ ]:
# sse = []
# r=range(1, 11)

# for k in r:
#     kmeans = KMeans(n_clusters=k)
#     kmeans.fit(cdf)
#     sse.append(kmeans.inertia_)
    
# plt.style.use("fivethirtyeight")
# plt.plot(r, sse)
# plt.xticks(r)
# plt.xlabel("Number of Clusters")
# plt.ylabel("SSE")
# plt.show()


In [ ]:
# kl = KneeLocator(range(1, 11), sse, curve="convex", direction="decreasing")
# kl.elbow
#import faiss

In [ ]:
#cdf=cdf[0:100000]

In [ ]:
arr=(cdf.values).copy(order='C').astype(np.float32)
ncentroids = 5000
n_init = 10
max_iter = 5
verbose = True

dim=cdf.shape[1]
dim

4

In [ ]:
%%time

kmeans = faiss.Kmeans(gpu=True,d=dim, k=ncentroids, niter=max_iter , verbose=verbose)
kmeans.train(arr)
D, I = kmeans.index.search(arr, 1)



CPU times: user 987 ms, sys: 744 ms, total: 1.73 s
Wall time: 1.67 s


In [ ]:
cdf['labels']=I
cdf['l2']=D

In [ ]:
x_train, x_valid, y_train, y_valid = model_selection.train_test_split(cdf.drop(columns=['GM%','labels']), cdf['labels'], test_size=0.25)


In [ ]:
%%time

from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier()
clf.fit(x_train,y_train)

CPU times: user 2min 10s, sys: 4.18 s, total: 2min 14s
Wall time: 2min 13s


In [ ]:
clf.feature_importances_

array([0.22078817, 0.33074731, 0.3642744 , 0.08419011])

In [ ]:
cdf['mingm']=cdf.groupby('labels')['GM%'].transform('min')
cdf['maxgm']=cdf.groupby('labels')['GM%'].transform('max')

In [ ]:
clf.score(x_valid,y_valid)

In [ ]:
cdf['F'] = cdf.groupby('labels')['GM%'].transform('min')
cdf['A'] = cdf.groupby('labels')['GM%'].transform('max')
cdf['dist']=(cdf['A']-cdf['F'])/4
cdf['D']=cdf['F']+1*cdf['dist']
cdf['C']=cdf['F']+2*cdf['dist']
cdf['B']=cdf['F']+3*cdf['dist']